In [1]:
import os
import spacy
import json
import string
import warnings
import tqdm
import re
import numpy as np
import pandas as pd

from pprint import pprint
from IPython.utils import io
from tqdm.notebook import tqdm
from gensim.models import Word2Vec 
from langdetect import DetectorFactory, detect
from IPython.core.display import HTML, display
from IPython.display import Image
from spacy.lang.en.stop_words import STOP_WORDS
import warnings
warnings.filterwarnings('ignore')

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
directory_path = r'C:\Users\ashok\Desktop\Chanu\Patadel Analytics Assignment\patent_jsons_ML Assignment\patent_jsons'

data_list = []

for filename in os.listdir(directory_path):
    if filename.endswith('.json'):
        file_path = os.path.join(directory_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
            data_list.append(json_data)
df = pd.DataFrame(data_list)

In [4]:
df.shape

(500, 21)

In [5]:
df.head()

,patent_number,publication_id,family_id,publication_date,titles,abstracts,claims,descriptions,inventors,assignees,...,locarno_classes,ipcr_classes,national_classes,ecla_classes,cpc_classes,f_term_classes,legal_status,priority_date,application_date,family_members
0,JP-H09311786-A,143280786,26393432,1997-12-02,"[{'lang': 'JA', 'text': 'データ処理装置'}, {'lang': '...","[{'lang': 'JA', 'paragraph_markup': '<abstract...","[{'lang': 'JA', 'claims': [{'num': 1, 'parent'...","[{'lang': 'EN', 'paragraph_markup': '<descript...","[{'first_name': '', 'last_name': 'UMINAGA MASA...",[],...,[],[{'label': 'G06F 9/38 20060101AFI2005...,"[{'primary': True, 'label': 'G06F 9/38 31...",[],[],"[{'label': '5B013 AA00'}, {'label': '5B013 AA1...",Pending,1996-03-18,1997-03-07,"[{'ucid': 'JP-H09311786-A', 'titles': [{'lang'..."
1,JP-H10177520-A,143364885,26550656,1998-06-30,"[{'lang': 'JA', 'text': 'データプロセッサ及びデータ処理システム'}...","[{'lang': 'JA', 'paragraph_markup': '<abstract...","[{'lang': 'EN', 'claims': [{'num': 1, 'parent'...","[{'lang': 'EN', 'paragraph_markup': '<descript...","[{'first_name': '', 'last_name': 'NISHIMOTO JU...",[],...,[],[{'label': 'G06F 12/10 20060101ALI2005...,"[{'primary': True, 'label': 'G06F 12/08 ...",[],[],"[{'label': '5B005 JJ11'}, {'label': '5B005 MM5...",Withdrawn,1996-10-16,1997-10-14,"[{'ucid': 'JP-H10177520-A', 'titles': [{'lang'..."
2,JP-H11184752-A,143645573,18409760,1999-07-09,"[{'lang': 'JA', 'text': 'データ処理装置及びデータ処理システム'},...","[{'lang': 'JA', 'paragraph_markup': '<abstract...","[{'lang': 'JA', 'claims': [{'num': 1, 'parent'...","[{'lang': 'JA', 'paragraph_markup': '<descript...","[{'first_name': '', 'last_name': 'TAMAKI SANEA...",[],...,[],[{'label': 'G06F 12/08 20060101AFI2005...,"[{'primary': True, 'label': 'G06F 12/08 ...",[{'label': 'Y02B60:12F'}],[{'label': 'Y02D 10/00'}],"[{'label': '5B005 JJ12'}, {'label': '5B005 UU3...",Withdrawn,1997-12-19,1997-12-19,"[{'ucid': 'JP-H11184752-A', 'titles': [{'lang'..."
3,US-20010011328-A1,75654482,25475553,2001-08-02,"[{'lang': 'EN', 'text': 'SOFTWARE-CONTROLLED C...","[{'lang': 'EN', 'paragraph_markup': '<abstract...","[{'lang': 'EN', 'claims': [{'num': 1, 'parent'...","[{'lang': 'EN', 'paragraph_markup': '<descript...","[{'first_name': 'Shine', 'last_name': 'Chung',...","[{'first_name': '', 'last_name': 'SONY CORPORA...",...,[],[{'label': 'G06F 12/12 20060101A I2005...,"[{'primary': True, 'label': '711128'}, {'prima...",[{'label': 'G06F 12/12B8'}],[{'label': 'G06F 12/128'}],[],Granted,1997-09-30,1997-09-30,"[{'ucid': 'US-20010011328-A1', 'titles': [{'la..."
4,US-20010049770-A1,75680368,22769873,2001-12-06,"[{'lang': 'EN', 'text': 'BUFFER MEMORY MANAGEM...","[{'lang': 'EN', 'paragraph_markup': '<abstract...","[{'lang': 'EN', 'claims': [{'num': 1, 'parent'...","[{'lang': 'EN', 'paragraph_markup': '<descript...","[{'first_name': 'Zhong-ning', 'last_name': 'Ca...","[{'first_name': '', 'last_name': 'INTEL CORPOR...",...,[],[{'label': 'G06F 12/08 20060101A I2005...,"[{'primary': True, 'label': '711129'}, {'prima...","[{'label': 'G06F 12/08B6M2'}, {'label': 'G06F...","[{'label': 'G06F 12/0848'}, {'label': 'G06F ...",[],Granted,1998-12-08,1998-12-08,"[{'ucid': 'AU-6497199-A', 'titles': [{'lang': ..."


In [6]:
# to check if all patents are unique

In [7]:
df['patent_number'].value_counts().reset_index()

,index,patent_number
0,JP-H09311786-A,1
1,US-6230279-B1,1
2,US-6256728-B1,1
3,US-6256727-B1,1
4,US-6253316-B1,1
...,...,...
495,US-5752261-A,1
496,US-5752255-A,1
497,US-5752069-A,1
498,US-5751996-A,1


In [8]:
# to get an idea about the dataset

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   patent_number     500 non-null    object
 1   publication_id    500 non-null    int64 
 2   family_id         500 non-null    int64 
 3   publication_date  500 non-null    object
 4   titles            500 non-null    object
 5   abstracts         500 non-null    object
 6   claims            500 non-null    object
 7   descriptions      500 non-null    object
 8   inventors         500 non-null    object
 9   assignees         500 non-null    object
 10  ipc_classes       500 non-null    object
 11  locarno_classes   500 non-null    object
 12  ipcr_classes      500 non-null    object
 13  national_classes  500 non-null    object
 14  ecla_classes      500 non-null    object
 15  cpc_classes       500 non-null    object
 16  f_term_classes    500 non-null    object
 17  legal_status    

In [10]:
# to filter patents which has more than language names

In [11]:
filtered_df = df[df['titles'].apply(len) >1]

In [12]:
filtered_df['titles'][0]

[{'lang': 'JA', 'text': 'データ処理装置'}, {'lang': 'EN', 'text': 'DATA PROCESSOR'}]

In [13]:
# getting only english titles

In [14]:
filtered_df['english_titles'] = filtered_df['titles'].apply(lambda titles: ' '.join([item['text'] for item in titles if item['lang'] == 'EN']))
filtered_df[['titles', 'english_titles']]

,titles,english_titles
0,"[{'lang': 'JA', 'text': 'データ処理装置'}, {'lang': '...",DATA PROCESSOR
1,"[{'lang': 'JA', 'text': 'データプロセッサ及びデータ処理システム'}...",DATA PROCESSOR AND DATA PROCESSING SYSTEM
2,"[{'lang': 'JA', 'text': 'データ処理装置及びデータ処理システム'},...",DATA PROCESSOR AND SYSTEM THEREFOR
22,"[{'lang': 'EN', 'text': 'Cache memory using a ...",Cache memory using a lowest priority replaceme...
24,"[{'lang': 'EN', 'text': 'I/O Storage controlle...",I/O Storage controller cache system with prefe...
25,"[{'lang': 'EN', 'text': 'In a data processing ...",In a data processing system Set associative se...
37,"[{'lang': 'EN', 'text': 'Retrieving requested ...",Retrieving requested data from a multi)set Cac...
38,"[{'lang': 'EN', 'text': 'Method and apparatus ...","Method and apparatus for indexing, accessing a..."
41,"[{'lang': 'EN', 'text': 'Computer system'}, {'...",Computer system Method for optimizing utilizat...
46,"[{'lang': 'EN', 'text': 'Storing device'}, {'l...",Storing device Data cache initialization


In [15]:
df['english_titles'] = df['titles'].apply(lambda titles: ' '.join([item['text'] for item in titles if item['lang'] == 'EN']))
english_records_df = df[df['english_titles'].apply(len) > 0]
english_records_df.head()

,patent_number,publication_id,family_id,publication_date,titles,abstracts,claims,descriptions,inventors,assignees,...,ipcr_classes,national_classes,ecla_classes,cpc_classes,f_term_classes,legal_status,priority_date,application_date,family_members,english_titles
0,JP-H09311786-A,143280786,26393432,1997-12-02,"[{'lang': 'JA', 'text': 'データ処理装置'}, {'lang': '...","[{'lang': 'JA', 'paragraph_markup': '<abstract...","[{'lang': 'JA', 'claims': [{'num': 1, 'parent'...","[{'lang': 'EN', 'paragraph_markup': '<descript...","[{'first_name': '', 'last_name': 'UMINAGA MASA...",[],...,[{'label': 'G06F 9/38 20060101AFI2005...,"[{'primary': True, 'label': 'G06F 9/38 31...",[],[],"[{'label': '5B013 AA00'}, {'label': '5B013 AA1...",Pending,1996-03-18,1997-03-07,"[{'ucid': 'JP-H09311786-A', 'titles': [{'lang'...",DATA PROCESSOR
1,JP-H10177520-A,143364885,26550656,1998-06-30,"[{'lang': 'JA', 'text': 'データプロセッサ及びデータ処理システム'}...","[{'lang': 'JA', 'paragraph_markup': '<abstract...","[{'lang': 'EN', 'claims': [{'num': 1, 'parent'...","[{'lang': 'EN', 'paragraph_markup': '<descript...","[{'first_name': '', 'last_name': 'NISHIMOTO JU...",[],...,[{'label': 'G06F 12/10 20060101ALI2005...,"[{'primary': True, 'label': 'G06F 12/08 ...",[],[],"[{'label': '5B005 JJ11'}, {'label': '5B005 MM5...",Withdrawn,1996-10-16,1997-10-14,"[{'ucid': 'JP-H10177520-A', 'titles': [{'lang'...",DATA PROCESSOR AND DATA PROCESSING SYSTEM
2,JP-H11184752-A,143645573,18409760,1999-07-09,"[{'lang': 'JA', 'text': 'データ処理装置及びデータ処理システム'},...","[{'lang': 'JA', 'paragraph_markup': '<abstract...","[{'lang': 'JA', 'claims': [{'num': 1, 'parent'...","[{'lang': 'JA', 'paragraph_markup': '<descript...","[{'first_name': '', 'last_name': 'TAMAKI SANEA...",[],...,[{'label': 'G06F 12/08 20060101AFI2005...,"[{'primary': True, 'label': 'G06F 12/08 ...",[{'label': 'Y02B60:12F'}],[{'label': 'Y02D 10/00'}],"[{'label': '5B005 JJ12'}, {'label': '5B005 UU3...",Withdrawn,1997-12-19,1997-12-19,"[{'ucid': 'JP-H11184752-A', 'titles': [{'lang'...",DATA PROCESSOR AND SYSTEM THEREFOR
3,US-20010011328-A1,75654482,25475553,2001-08-02,"[{'lang': 'EN', 'text': 'SOFTWARE-CONTROLLED C...","[{'lang': 'EN', 'paragraph_markup': '<abstract...","[{'lang': 'EN', 'claims': [{'num': 1, 'parent'...","[{'lang': 'EN', 'paragraph_markup': '<descript...","[{'first_name': 'Shine', 'last_name': 'Chung',...","[{'first_name': '', 'last_name': 'SONY CORPORA...",...,[{'label': 'G06F 12/12 20060101A I2005...,"[{'primary': True, 'label': '711128'}, {'prima...",[{'label': 'G06F 12/12B8'}],[{'label': 'G06F 12/128'}],[],Granted,1997-09-30,1997-09-30,"[{'ucid': 'US-20010011328-A1', 'titles': [{'la...",SOFTWARE-CONTROLLED CACHE MEMORY COMPARTMENTAL...
4,US-20010049770-A1,75680368,22769873,2001-12-06,"[{'lang': 'EN', 'text': 'BUFFER MEMORY MANAGEM...","[{'lang': 'EN', 'paragraph_markup': '<abstract...","[{'lang': 'EN', 'claims': [{'num': 1, 'parent'...","[{'lang': 'EN', 'paragraph_markup': '<descript...","[{'first_name': 'Zhong-ning', 'last_name': 'Ca...","[{'first_name': '', 'last_name': 'INTEL CORPOR...",...,[{'label': 'G06F 12/08 20060101A I2005...,"[{'primary': True, 'label': '711129'}, {'prima...","[{'label': 'G06F 12/08B6M2'}, {'label': 'G06F...","[{'label': 'G06F 12/0848'}, {'label': 'G06F ...",[],Granted,1998-12-08,1998-12-08,"[{'ucid': 'AU-6497199-A', 'titles': [{'lang': ...",BUFFER MEMORY MANAGEMENT IN A SYSTEM HAVING MU...


In [16]:
# Extracting only english language text from all the patents

In [17]:
english_records_df['abstracts'][1]

[{'lang': 'JA',
  'paragraph_markup': '<abstract lang="JA" load-source="patent-office" mxw-id="PA58971002"><p>(57)【要約】\n【課題】  アドレス変換動作を高速化したデータプロセ<br/>ッサを提供する。\n【解決手段】  アドレス変換バッファをデータ用と命令<br/>用に分け、データ用のアドレス変換バッファ（４）には<br/>命令用のアドレス変換情報も格納させ、命令用のアドレ<br/>ス変換バッファ（３）で変換ミスを生じたときは、新た<br/>なアドレス変換情報をデータ用のアドレス変換バッファ<br/>（４）から取り込むようにする。命令用のアドレス変換<br/>バッファで変換ミスを生じたとき毎回外部のアドレス変<br/>換テーブルからアドレス変換情報を取得する場合に比べ<br/>て、アドレス変換動作の高速化を実現できる。\n</p></abstract>'},
 {'lang': 'EN',
  'paragraph_markup': '<abstract lang="EN" load-source="docdb" mxw-id="PA94035622" source="PAJ"><p>PROBLEM TO BE SOLVED: To provide a data processor which is improved in address converting speed. SOLUTION: The address translation buffer of this data processor is divided into an address translation buffer 4 for data and an address translation buffer 3 for instruction so that address translation information for instruction can also be stored in the buffer 4 and, when a translation error occurs in the buffer 3, new address translation in

In [18]:
def modify_abstract(abstracts):
    english_abstracts = [re.sub(r'<.*?>', '', item['paragraph_markup']) for item in abstracts if item['lang'] == 'EN']
    priority_sources = ['patent-office', 'google', 'docdb']

    for source in priority_sources:
        for abstract in english_abstracts:
            if source in abstract:
                return abstract
    
    if english_abstracts:
        return english_abstracts[0]
    
    return None 

df['modified_abstract'] = df['abstracts'].apply(modify_abstract)

df.head()


,patent_number,publication_id,family_id,publication_date,titles,abstracts,claims,descriptions,inventors,assignees,...,national_classes,ecla_classes,cpc_classes,f_term_classes,legal_status,priority_date,application_date,family_members,english_titles,modified_abstract
0,JP-H09311786-A,143280786,26393432,1997-12-02,"[{'lang': 'JA', 'text': 'データ処理装置'}, {'lang': '...","[{'lang': 'JA', 'paragraph_markup': '<abstract...","[{'lang': 'JA', 'claims': [{'num': 1, 'parent'...","[{'lang': 'EN', 'paragraph_markup': '<descript...","[{'first_name': '', 'last_name': 'UMINAGA MASA...",[],...,"[{'primary': True, 'label': 'G06F 9/38 31...",[],[],"[{'label': '5B013 AA00'}, {'label': '5B013 AA1...",Pending,1996-03-18,1997-03-07,"[{'ucid': 'JP-H09311786-A', 'titles': [{'lang'...",DATA PROCESSOR,PROBLEM TO BE SOLVED: To reduce a pipeline sta...
1,JP-H10177520-A,143364885,26550656,1998-06-30,"[{'lang': 'JA', 'text': 'データプロセッサ及びデータ処理システム'}...","[{'lang': 'JA', 'paragraph_markup': '<abstract...","[{'lang': 'EN', 'claims': [{'num': 1, 'parent'...","[{'lang': 'EN', 'paragraph_markup': '<descript...","[{'first_name': '', 'last_name': 'NISHIMOTO JU...",[],...,"[{'primary': True, 'label': 'G06F 12/08 ...",[],[],"[{'label': '5B005 JJ11'}, {'label': '5B005 MM5...",Withdrawn,1996-10-16,1997-10-14,"[{'ucid': 'JP-H10177520-A', 'titles': [{'lang'...",DATA PROCESSOR AND DATA PROCESSING SYSTEM,PROBLEM TO BE SOLVED: To provide a data proces...
2,JP-H11184752-A,143645573,18409760,1999-07-09,"[{'lang': 'JA', 'text': 'データ処理装置及びデータ処理システム'},...","[{'lang': 'JA', 'paragraph_markup': '<abstract...","[{'lang': 'JA', 'claims': [{'num': 1, 'parent'...","[{'lang': 'JA', 'paragraph_markup': '<descript...","[{'first_name': '', 'last_name': 'TAMAKI SANEA...",[],...,"[{'primary': True, 'label': 'G06F 12/08 ...",[{'label': 'Y02B60:12F'}],[{'label': 'Y02D 10/00'}],"[{'label': '5B005 JJ12'}, {'label': '5B005 UU3...",Withdrawn,1997-12-19,1997-12-19,"[{'ucid': 'JP-H11184752-A', 'titles': [{'lang'...",DATA PROCESSOR AND SYSTEM THEREFOR,PROBLEM TO BE SOLVED: To maintain the equivale...
3,US-20010011328-A1,75654482,25475553,2001-08-02,"[{'lang': 'EN', 'text': 'SOFTWARE-CONTROLLED C...","[{'lang': 'EN', 'paragraph_markup': '<abstract...","[{'lang': 'EN', 'claims': [{'num': 1, 'parent'...","[{'lang': 'EN', 'paragraph_markup': '<descript...","[{'first_name': 'Shine', 'last_name': 'Chung',...","[{'first_name': '', 'last_name': 'SONY CORPORA...",...,"[{'primary': True, 'label': '711128'}, {'prima...",[{'label': 'G06F 12/12B8'}],[{'label': 'G06F 12/128'}],[],Granted,1997-09-30,1997-09-30,"[{'ucid': 'US-20010011328-A1', 'titles': [{'la...",SOFTWARE-CONTROLLED CACHE MEMORY COMPARTMENTAL...,A method and apparatus for controlling compart...
4,US-20010049770-A1,75680368,22769873,2001-12-06,"[{'lang': 'EN', 'text': 'BUFFER MEMORY MANAGEM...","[{'lang': 'EN', 'paragraph_markup': '<abstract...","[{'lang': 'EN', 'claims': [{'num': 1, 'parent'...","[{'lang': 'EN', 'paragraph_markup': '<descript...","[{'first_name': 'Zhong-ning', 'last_name': 'Ca...","[{'first_name': '', 'last_name': 'INTEL CORPOR...",...,"[{'primary': True, 'label': '711129'}, {'prima...","[{'label': 'G06F 12/08B6M2'}, {'label': 'G06F...","[{'label': 'G06F 12/0848'}, {'label': 'G06F ...",[],Granted,1998-12-08,1998-12-08,"[{'ucid': 'AU-6497199-A', 'titles': [{'lang': ...",BUFFER MEMORY MANAGEMENT IN A SYSTEM HAVING MU...,A system includes multiple program execution e...


In [19]:
df.modified_abstract[0]

'PROBLEM TO BE SOLVED: To reduce a pipeline stall due to a data hazard of a superscalar system and to improve the processing speed by changing an instruction in 1st instruction format stored in an instruction memory into an instruction in 2nd instruction format. SOLUTION: The instruction is taken in a 1st stage from the instruction memory and the instruction taken in the 1st stage 101 is decoded in a 2nd stage 103. The decoded instruction is executed in a 3rd stage and when the execution result is written in a register in a 4th stage 107, the instruction in the 1st instruction format stored in the instruction memory is changed into the instruction in the 2nd instruction format and executed. Consequently, the pipeline stall due to the data hazard of the superscalar system can be reduced and the processing speed is improved.'

In [20]:
df['claims'][1]

[{'lang': 'EN',
  'claims': [{'num': 1,
    'parent': -1,
    'type': 'independent',
    'paragraph_markup': '<claim num="1"><claim-text>1. A central processing unit, and a part of address conversion information for converting a virtual address output from the central processing unit into a physical address is stored, and a physical address corresponding to the virtual address is associated with the address conversion information. A first address translation buffer to be searched, and address translation information relating to an instruction address in the address translation information held by the first address translation buffer. An address is associatively searched from the address conversion information, and when the result of the associative search is a search error, the first address is determined by the virtual address related to the search error. <br/> A second address translation buffer for causing the address translation buffer to perform an associative search and acquiring

In [21]:
# Below code extracts english claims only and removes any tags present

In [22]:
def modify_claims(claims):
    english_claims = []
    for item in claims:
        if item['lang'] == 'EN':
            for claim_item in item['claims']:
                english_claims.append(re.sub(r'<.*?>', '', claim_item['paragraph_markup']))
    return english_claims

df['english_claims'] = df['claims'].apply(modify_claims)
df.head()


,patent_number,publication_id,family_id,publication_date,titles,abstracts,claims,descriptions,inventors,assignees,...,ecla_classes,cpc_classes,f_term_classes,legal_status,priority_date,application_date,family_members,english_titles,modified_abstract,english_claims
0,JP-H09311786-A,143280786,26393432,1997-12-02,"[{'lang': 'JA', 'text': 'データ処理装置'}, {'lang': '...","[{'lang': 'JA', 'paragraph_markup': '<abstract...","[{'lang': 'JA', 'claims': [{'num': 1, 'parent'...","[{'lang': 'EN', 'paragraph_markup': '<descript...","[{'first_name': '', 'last_name': 'UMINAGA MASA...",[],...,[],[],"[{'label': '5B013 AA00'}, {'label': '5B013 AA1...",Pending,1996-03-18,1997-03-07,"[{'ucid': 'JP-H09311786-A', 'titles': [{'lang'...",DATA PROCESSOR,PROBLEM TO BE SOLVED: To reduce a pipeline sta...,[1. A data processing device that divides into...
1,JP-H10177520-A,143364885,26550656,1998-06-30,"[{'lang': 'JA', 'text': 'データプロセッサ及びデータ処理システム'}...","[{'lang': 'JA', 'paragraph_markup': '<abstract...","[{'lang': 'EN', 'claims': [{'num': 1, 'parent'...","[{'lang': 'EN', 'paragraph_markup': '<descript...","[{'first_name': '', 'last_name': 'NISHIMOTO JU...",[],...,[],[],"[{'label': '5B005 JJ11'}, {'label': '5B005 MM5...",Withdrawn,1996-10-16,1997-10-14,"[{'ucid': 'JP-H10177520-A', 'titles': [{'lang'...",DATA PROCESSOR AND DATA PROCESSING SYSTEM,PROBLEM TO BE SOLVED: To provide a data proces...,"[1. A central processing unit, and a part of a..."
2,JP-H11184752-A,143645573,18409760,1999-07-09,"[{'lang': 'JA', 'text': 'データ処理装置及びデータ処理システム'},...","[{'lang': 'JA', 'paragraph_markup': '<abstract...","[{'lang': 'JA', 'claims': [{'num': 1, 'parent'...","[{'lang': 'JA', 'paragraph_markup': '<descript...","[{'first_name': '', 'last_name': 'TAMAKI SANEA...",[],...,[{'label': 'Y02B60:12F'}],[{'label': 'Y02D 10/00'}],"[{'label': '5B005 JJ12'}, {'label': '5B005 UU3...",Withdrawn,1997-12-19,1997-12-19,"[{'ucid': 'JP-H11184752-A', 'titles': [{'lang'...",DATA PROCESSOR AND SYSTEM THEREFOR,PROBLEM TO BE SOLVED: To maintain the equivale...,[1. An access control means for outputting an ...
3,US-20010011328-A1,75654482,25475553,2001-08-02,"[{'lang': 'EN', 'text': 'SOFTWARE-CONTROLLED C...","[{'lang': 'EN', 'paragraph_markup': '<abstract...","[{'lang': 'EN', 'claims': [{'num': 1, 'parent'...","[{'lang': 'EN', 'paragraph_markup': '<descript...","[{'first_name': 'Shine', 'last_name': 'Chung',...","[{'first_name': '', 'last_name': 'SONY CORPORA...",...,[{'label': 'G06F 12/12B8'}],[{'label': 'G06F 12/128'}],[],Granted,1997-09-30,1997-09-30,"[{'ucid': 'US-20010011328-A1', 'titles': [{'la...",SOFTWARE-CONTROLLED CACHE MEMORY COMPARTMENTAL...,A method and apparatus for controlling compart...,[1. A cache memory comprising: \na plurality o...
4,US-20010049770-A1,75680368,22769873,2001-12-06,"[{'lang': 'EN', 'text': 'BUFFER MEMORY MANAGEM...","[{'lang': 'EN', 'paragraph_markup': '<abstract...","[{'lang': 'EN', 'claims': [{'num': 1, 'parent'...","[{'lang': 'EN', 'paragraph_markup': '<descript...","[{'first_name': 'Zhong-ning', 'last_name': 'Ca...","[{'first_name': '', 'last_name': 'INTEL CORPOR...",...,"[{'label': 'G06F 12/08B6M2'}, {'label': 'G06F...","[{'label': 'G06F 12/0848'}, {'label': 'G06F ...",[],Granted,1998-12-08,1998-12-08,"[{'ucid': 'AU-6497199-A', 'titles': [{'lang': ...",BUFFER MEMORY MANAGEMENT IN A SYSTEM HAVING MU...,A system includes multiple program execution e...,[1. A system comprising: \na processor; a plur...


In [23]:
# Below code extracts english description and ignore other languages and tags

In [24]:
def modify_description(descriptions):
    english_description = ""
    for item in descriptions:
        if item['lang'] == 'EN':
            english_description = re.sub(r'<.*?>', '', item['paragraph_markup'])
            break 
    return english_description

df['english_description'] = df['descriptions'].apply(modify_description)

In [25]:
df.english_description[0]

'Detailed Description of the Invention[0001]BACKGROUND OF THE INVENTION 1. Field of the Invention The present invention relates to a data processing device such as a microprocessor or a microcomputer, and more particularly to a technique effective when applied to a data processing device such as superscalar for parallel processing.[0002]2. Description of the Related Art A microprocessor (CPU (Central P  rocessing unit), microcomputer, etc. ) Sequentially fetches a sequence of instructions,  Decrypt and execute. The instructions executed by the microprocessor are now of fixed length with the aim of simplifying the decoding circuit. A microprocessor that executes fixed-length instructions in a pipeline (Pipelining)  It is called an ISC (Red used Instruction Set Computer) type processor.FIG. 1 shows a pipelined implementation method of a microprocessor. Here, for simplification, the normally existing memory access stage (M  EM) is omitted. Individual stages (101, 10)  3, 105, 107) is a un

In [26]:
df.family_members[12]

[{'ucid': 'CA-1129113-A1', 'titles': []},
 {'ucid': 'JP-S5939831-B2', 'titles': []},
 {'ucid': 'JP-1263896-C', 'titles': []},
 {'ucid': 'YU-39513-B1', 'titles': []},
 {'ucid': 'YU-302078-A1', 'titles': []},
 {'ucid': 'YU-39513-B',
  'titles': [{'lang': 'EN',
    'text': 'OUT-OF-STORAGE INDICATOR FOR A SECUNDARY SORAGE IN A TEST MANNER OPERATION'}]},
 {'ucid': 'YU-302078-A',
  'titles': [{'lang': 'EN',
    'text': 'SYSTEM OF AN OUT-OF-STORE INDICATOR FOR SECONDARY STORES IN TEST OPERATING MANNER'}]},
 {'ucid': 'JP-S5496938-A',
  'titles': [{'lang': 'EN',
    'text': 'DATA PROCESSING SYSTEM HAVING CACHE STORAGE INITIALIZATION FUNCTION'}]},
 {'ucid': 'US-4195341-A',
  'titles': [{'lang': 'EN',
    'text': 'Initialization of cache store to assure valid data'}]},
 {'ucid': 'AU-4288478-A',
  'titles': [{'lang': 'EN', 'text': 'INITIALIZATION OF CACHE STORE'}]},
 {'ucid': 'CA-1129113-A',
  'titles': [{'lang': 'FR', 'text': "INITIALISATION D'UNE ANTEMEMOIRE"},
   {'lang': 'EN', 'text': 'INITIAL

In [27]:
# Below code will extract only english language titles and ucid

In [28]:
def process_family_members(row):
    ucid_trimmed = [entry['ucid'] for entry in row]
    english_titles = [title['text'] for entry in row for title in entry['titles'] if title['lang'] == 'EN']
    return pd.Series([ucid_trimmed, english_titles], index=['ucid_trimmed', 'english_family_members'])

df[['ucid_trimmed', 'english_family_members']] = df['family_members'].apply(process_family_members)

In [29]:
df.english_family_members[12]

['OUT-OF-STORAGE INDICATOR FOR A SECUNDARY SORAGE IN A TEST MANNER OPERATION',
 'SYSTEM OF AN OUT-OF-STORE INDICATOR FOR SECONDARY STORES IN TEST OPERATING MANNER',
 'DATA PROCESSING SYSTEM HAVING CACHE STORAGE INITIALIZATION FUNCTION',
 'Initialization of cache store to assure valid data',
 'INITIALIZATION OF CACHE STORE',
 'INITIALIZATION OF CACHE STORE',
 'INITIALIZATION OF CACHE STORE']

In [30]:
# After performing all the preprocessing steps

In [31]:
df.shape

(500, 27)

In [32]:
df.columns

Index(['patent_number', 'publication_id', 'family_id', 'publication_date',
       'titles', 'abstracts', 'claims', 'descriptions', 'inventors',
       'assignees', 'ipc_classes', 'locarno_classes', 'ipcr_classes',
       'national_classes', 'ecla_classes', 'cpc_classes', 'f_term_classes',
       'legal_status', 'priority_date', 'application_date', 'family_members',
       'english_titles', 'modified_abstract', 'english_claims',
       'english_description', 'ucid_trimmed', 'english_family_members'],
      dtype='object')

In [33]:
# Creating a new dataframe with only required columns

In [34]:
cleaned_df = df[['patent_number','english_titles','modified_abstract','english_claims','english_description','english_family_members']]

In [35]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   patent_number           500 non-null    object
 1   english_titles          500 non-null    object
 2   modified_abstract       500 non-null    object
 3   english_claims          500 non-null    object
 4   english_description     500 non-null    object
 5   english_family_members  500 non-null    object
dtypes: object(6)
memory usage: 23.6+ KB


In [36]:
# Dropping Duplicates

In [37]:
cleaned_df.drop_duplicates(['modified_abstract'], inplace=True)
cleaned_df.dropna(inplace=True)
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490 entries, 0 to 499
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   patent_number           490 non-null    object
 1   english_titles          490 non-null    object
 2   modified_abstract       490 non-null    object
 3   english_claims          490 non-null    object
 4   english_description     490 non-null    object
 5   english_family_members  490 non-null    object
dtypes: object(6)
memory usage: 26.8+ KB


In [38]:
punctuations = string.punctuation
stopwords = list(STOP_WORDS)

In [39]:
# Converting features from list to string and removing stop words

In [40]:
cleaned_df.head()

,patent_number,english_titles,modified_abstract,english_claims,english_description,english_family_members
0,JP-H09311786-A,DATA PROCESSOR,PROBLEM TO BE SOLVED: To reduce a pipeline sta...,[1. A data processing device that divides into...,Detailed Description of the Invention[0001]BAC...,[DATA PROCESSOR]
1,JP-H10177520-A,DATA PROCESSOR AND DATA PROCESSING SYSTEM,PROBLEM TO BE SOLVED: To provide a data proces...,"[1. A central processing unit, and a part of a...",DETAILED DESCRIPTION OF THE INVENTION[0001]BAC...,[DATA PROCESSOR AND DATA PROCESSING SYSTEM]
2,JP-H11184752-A,DATA PROCESSOR AND SYSTEM THEREFOR,PROBLEM TO BE SOLVED: To maintain the equivale...,[1. An access control means for outputting an ...,DETAILED DESCRIPTION OF THE INVENTION[0001]BAC...,[DATA PROCESSOR AND SYSTEM THEREFOR]
3,US-20010011328-A1,SOFTWARE-CONTROLLED CACHE MEMORY COMPARTMENTAL...,A method and apparatus for controlling compart...,[1. A cache memory comprising: \na plurality o...,FIELD OF THE INVENTION [0001] The present inve...,[SOFTWARE-CONTROLLED CACHE MEMORY COMPARTMENTA...
4,US-20010049770-A1,BUFFER MEMORY MANAGEMENT IN A SYSTEM HAVING MU...,A system includes multiple program execution e...,[1. A system comprising: \na processor; a plur...,BACKGROUND [0001] The invention relates to buf...,[Buffer memory management in a system having m...


In [41]:
cleaned_df['english_claims'] = cleaned_df['english_claims'].apply(lambda claims: ' '.join(re.sub(r'^\d+\.\s*', '', claim) for claim in claims))

In [42]:
cleaned_df.head()

,patent_number,english_titles,modified_abstract,english_claims,english_description,english_family_members
0,JP-H09311786-A,DATA PROCESSOR,PROBLEM TO BE SOLVED: To reduce a pipeline sta...,A data processing device that divides into a p...,Detailed Description of the Invention[0001]BAC...,[DATA PROCESSOR]
1,JP-H10177520-A,DATA PROCESSOR AND DATA PROCESSING SYSTEM,PROBLEM TO BE SOLVED: To provide a data proces...,"A central processing unit, and a part of addre...",DETAILED DESCRIPTION OF THE INVENTION[0001]BAC...,[DATA PROCESSOR AND DATA PROCESSING SYSTEM]
2,JP-H11184752-A,DATA PROCESSOR AND SYSTEM THEREFOR,PROBLEM TO BE SOLVED: To maintain the equivale...,An access control means for outputting an addr...,DETAILED DESCRIPTION OF THE INVENTION[0001]BAC...,[DATA PROCESSOR AND SYSTEM THEREFOR]
3,US-20010011328-A1,SOFTWARE-CONTROLLED CACHE MEMORY COMPARTMENTAL...,A method and apparatus for controlling compart...,A cache memory comprising: \na plurality of st...,FIELD OF THE INVENTION [0001] The present inve...,[SOFTWARE-CONTROLLED CACHE MEMORY COMPARTMENTA...
4,US-20010049770-A1,BUFFER MEMORY MANAGEMENT IN A SYSTEM HAVING MU...,A system includes multiple program execution e...,A system comprising: \na processor; a pluralit...,BACKGROUND [0001] The invention relates to buf...,[Buffer memory management in a system having m...


In [43]:
cleaned_df['english_family_members'][10]

['OUT OF STORE INDICATOR FOR A CACHE  STORE IN A TEST MODE',
 'SYSTEM FOR INITIALIZING THE CACHE STORAGE',
 'Out of store indicator for a cache store in test mode',
 'DATA PROCESSING SYSTEM HAVING CACHE STORAGE NO MEMORY DISPLAY UNIT IN TEST MODE']

In [44]:
cleaned_df['english_family_members'] = cleaned_df['english_family_members'].apply(lambda sentences: ' '.join(set(' '.join(word.lower() for word in sentence.split() if word.lower() not in stopwords) for sentence in sentences)))

In [45]:
cleaned_df['english_family_members'][10]

'store indicator cache store test mode system initializing cache storage data processing system having cache storage memory display unit test mode'

In [46]:
cleaned_df.head()

,patent_number,english_titles,modified_abstract,english_claims,english_description,english_family_members
0,JP-H09311786-A,DATA PROCESSOR,PROBLEM TO BE SOLVED: To reduce a pipeline sta...,A data processing device that divides into a p...,Detailed Description of the Invention[0001]BAC...,data processor
1,JP-H10177520-A,DATA PROCESSOR AND DATA PROCESSING SYSTEM,PROBLEM TO BE SOLVED: To provide a data proces...,"A central processing unit, and a part of addre...",DETAILED DESCRIPTION OF THE INVENTION[0001]BAC...,data processor data processing system
2,JP-H11184752-A,DATA PROCESSOR AND SYSTEM THEREFOR,PROBLEM TO BE SOLVED: To maintain the equivale...,An access control means for outputting an addr...,DETAILED DESCRIPTION OF THE INVENTION[0001]BAC...,data processor system therefor
3,US-20010011328-A1,SOFTWARE-CONTROLLED CACHE MEMORY COMPARTMENTAL...,A method and apparatus for controlling compart...,A cache memory comprising: \na plurality of st...,FIELD OF THE INVENTION [0001] The present inve...,software-controlled cache memory compartmental...
4,US-20010049770-A1,BUFFER MEMORY MANAGEMENT IN A SYSTEM HAVING MU...,A system includes multiple program execution e...,A system comprising: \na processor; a pluralit...,BACKGROUND [0001] The invention relates to buf...,buffer memory management system haivng multipl...


In [47]:
# cleaned_df.to_csv("cleaned_df.csv")

In [48]:
cleaned_df.to_csv('cleaned_df.zip', index=False, compression=dict(method='zip', archive_name='cleaned_df.csv'))
